In [2]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

# Basic

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.graph import END, MessageGraph

model = ChatOpenAI(temperature=0, base_url='https://api.deepseek.com', model='deepseek-chat')

graph = MessageGraph()

graph.add_node("oracle", model)
graph.add_edge("oracle", END)

graph.set_entry_point("oracle")

runnable = graph.compile()
runnable

CompiledStateGraph(nodes={'__start__': PregelNode(config={'tags': ['langsmith:hidden']}, channels=['__start__'], triggers=['__start__'], writers=[ChannelWrite<__root__>(recurse=True, writes=[ChannelWriteEntry(channel='__root__', value=<object object at 0x7f50c8fc1a60>, skip_none=True, mapper=None)]), ChannelWrite<start:oracle>(recurse=True, writes=[ChannelWriteEntry(channel='start:oracle', value='__start__', skip_none=False, mapper=None)])]), 'oracle': PregelNode(config={'tags': []}, channels=['__root__'], triggers=['start:oracle'], writers=[ChannelWrite<oracle,__root__>(recurse=True, writes=[ChannelWriteEntry(channel='oracle', value='oracle', skip_none=False, mapper=None), ChannelWriteEntry(channel='__root__', value=<object object at 0x7f50c8fc1a60>, skip_none=True, mapper=None)])])}, channels={'__root__': <langgraph.channels.binop.BinaryOperatorAggregate object at 0x7f50b2bd2320>, '__start__': <langgraph.channels.ephemeral_value.EphemeralValue object at 0x7f50b3a602b0>, 'oracle': <la

In [4]:
runnable.invoke(HumanMessage("What is 1 + 1?"))

[HumanMessage(content='What is 1 + 1?', id='344f159c-fd73-4b6b-a0c2-b14491c50a18'),
 AIMessage(content=' 1 + 1 equals 2.', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 15, 'total_tokens': 24}, 'model_name': 'deepseek-chat', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b4a4796c-ffd2-44c3-8dcf-4fdce1d4f55b-0')]

# 条件分支

In [5]:
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

@tool
def multiply(first_number: int, second_number: int):
    """Multiplies two numbers together."""
    return first_number * second_number

model = ChatOpenAI(temperature=0, base_url='https://api.deepseek.com', model='deepseek-chat')
model_with_tools = model.bind_tools([multiply])

builder = MessageGraph()

builder.add_node("oracle", model_with_tools)

tool_node = ToolNode([multiply])
builder.add_node("multiply", tool_node)

builder.add_edge("multiply", END)

builder.set_entry_point("oracle")

In [6]:
from typing import Literal

# state: List[BaseMessage]
def router(state) -> Literal["multiply", "__end__"]:
    tool_calls = state[-1].additional_kwargs.get("tool_calls", [])
    if len(tool_calls):
        return "multiply"
    else:
        return "__end__"

builder.add_conditional_edges("oracle", router)

In [9]:
runnable = builder.compile()

runnable.invoke(HumanMessage("What is 45343 * 45343 ? I would prefer you to call the tool."))

[HumanMessage(content='What is 45343 * 45343 * 456? I would prefer you to call the tool.', id='c3c0c4ee-c112-4f8c-bc1f-eeef93c60507'),
 AIMessage(content=" To calculate the expression \\( 45343 \\times 45343 \\times 456 \\), we can use a calculator or a computational tool. Here's the calculation:\n\n1. Calculate \\( 45343 \\times 45343 \\):\n   \\[ 45343 \\times 45343 = 2055998649 \\]\n\n2. Multiply the result by \\( 456 \\):\n   \\[ 2055998649 \\times 456 = 938035389044 \\]\n\nThus, the result of \\( 45343 \\times 45343 \\times 456 \\) is \\( 938035389044 \\).", response_metadata={'token_usage': {'completion_tokens': 192, 'prompt_tokens': 37, 'total_tokens': 229}, 'model_name': 'deepseek-chat', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-19bb5ff1-0ac4-4d7e-88f4-92390ffe70e2-0')]